In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [128]:
merged_data = pd.read_csv('Merged_Data.csv')
md = merged_data
md.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [130]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    1000 non-null   object 
 1   CustomerID       1000 non-null   object 
 2   ProductID        1000 non-null   object 
 3   TransactionDate  1000 non-null   object 
 4   Quantity         1000 non-null   int64  
 5   TotalValue       1000 non-null   float64
 6   Price_x          1000 non-null   float64
 7   CustomerName     1000 non-null   object 
 8   Region           1000 non-null   object 
 9   SignupDate       1000 non-null   object 
 10  ProductName      1000 non-null   object 
 11  Category         1000 non-null   object 
 12  Price_y          1000 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 101.7+ KB


In [132]:
md['TransactionDate'] = pd.to_datetime(md['TransactionDate'], errors = 'coerce')
md['SignupDate'] = pd.to_datetime(md['SignupDate'], errors = 'coerce')

In [134]:
md.sample()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
194,T00432,C0065,P038,2024-12-13 21:18:19,3,441.66,147.22,Gerald Hines,North America,2024-07-10,TechPro Vase,Home Decor,147.22


In [136]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   TransactionID    1000 non-null   object        
 1   CustomerID       1000 non-null   object        
 2   ProductID        1000 non-null   object        
 3   TransactionDate  1000 non-null   datetime64[ns]
 4   Quantity         1000 non-null   int64         
 5   TotalValue       1000 non-null   float64       
 6   Price_x          1000 non-null   float64       
 7   CustomerName     1000 non-null   object        
 8   Region           1000 non-null   object        
 9   SignupDate       1000 non-null   datetime64[ns]
 10  ProductName      1000 non-null   object        
 11  Category         1000 non-null   object        
 12  Price_y          1000 non-null   float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(7)
memory usage: 101.7+ KB


In [138]:
md['CustomerTenure'] = (md['TransactionDate'].max() - md['SignupDate']).dt.days

In [140]:
customer_features = md.groupby('CustomerID').agg({
    'Region': 'first',  # The customer's region (e.g., Asia, Europe)
    'TotalValue': 'sum',  # Total money spent
    'TransactionID': 'count',  # Total transactions made
    'ProductID': pd.Series.nunique,  # Unique products purchased
    'Category': pd.Series.nunique,  # Unique categories purchased
    'CustomerTenure': 'mean'  # Average number of days since signup
}).reset_index()

In [142]:
customer_features.rename(columns = {
    'TotalValue': 'Total_Spend',
    'TransactionID': 'Transaction_Count',
    'ProductID': 'Unique_Products',
    'Category': 'Unique_Categories',
    'CustomerTenure': 'Customer_Tenure'
}, inplace = True)

In [144]:
customer_features.head()

,CustomerID,Region,Total_Spend,Transaction_Count,Unique_Products,Unique_Categories,Customer_Tenure
0,C0001,South America,3354.52,5,5,3,902.0
1,C0002,Asia,1862.74,4,4,2,1049.0
2,C0003,South America,2725.38,4,4,3,296.0
3,C0004,South America,5354.88,8,8,3,811.0
4,C0005,Asia,2034.24,3,3,2,866.0


In [146]:
customer_features = pd.get_dummies(customer_features, columns = ['Region'], dtype = int)

In [148]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
numerical_columns = ['Total_Spend', 'Transaction_Count', 'Unique_Products', 'Unique_Categories', 'Customer_Tenure']
customer_features[numerical_columns] = scaler.fit_transform(customer_features[numerical_columns])

In [150]:
customer_features.head()

,CustomerID,Total_Spend,Transaction_Count,Unique_Products,Unique_Categories,Customer_Tenure,Region_Asia,Region_Europe,Region_North America,Region_South America
0,C0001,0.308942,0.4,0.444444,0.666667,0.842204,0,0,0,1
1,C0002,0.168095,0.3,0.333333,0.333333,0.979458,1,0,0,0
2,C0003,0.249541,0.3,0.333333,0.666667,0.276377,0,0,0,1
3,C0004,0.497806,0.7,0.777778,0.666667,0.757236,0,0,0,1
4,C0005,0.184287,0.2,0.222222,0.333333,0.808590,1,0,0,0


In [152]:
customer_features.to_csv('Customer_Features.csv', index=False)
from IPython.display import FileLink
FileLink('Customer_Features.csv')

/Users/adityagarg/Zeotap/Customer_Features.csv

In [154]:
from sklearn.metrics.pairwise import cosine_similarity

features_for_similarity = customer_features.drop(columns=['CustomerID'])
similarity_matrix = cosine_similarity(features_for_similarity)

# Convert the similarity matrix into a DataFrame for easy access:
similarity_df = pd.DataFrame(similarity_matrix, index = customer_features['CustomerID'],
                             columns = customer_features['CustomerID'])

In [156]:
similarity_df.head()

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.558392,0.937794,0.966857,0.513349,0.947364,0.525693,0.573434,0.425033,0.515832,...,0.915606,0.993743,0.427515,0.538247,0.881214,0.596685,0.472680,0.494734,0.520260,0.590996
C0002,0.558392,1.000000,0.363326,0.545614,0.992434,0.388794,0.992980,0.432336,0.396020,0.498054,...,0.347559,0.531131,0.952667,0.386962,0.320729,0.543655,0.452456,0.521166,0.502158,0.859464
C0003,0.937794,0.363326,1.000000,0.917724,0.340920,0.994274,0.345951,0.541213,0.316601,0.362981,...,0.961523,0.944153,0.258102,0.528736,0.969291,0.451643,0.335912,0.299632,0.365725,0.529882
C0004,0.966857,0.545614,0.917724,1.000000,0.491885,0.931793,0.503604,0.680794,0.424019,0.523574,...,0.935163,0.936639,0.459527,0.616479,0.923969,0.585955,0.466195,0.435752,0.528515,0.619902
C0005,0.513349,0.992434,0.340920,0.491885,1.000000,0.367193,0.998979,0.386788,0.361769,0.450777,...,0.310514,0.491295,0.954578,0.356986,0.293453,0.510668,0.415688,0.481082,0.455138,0.873897


In [158]:
target_customers = customer_features['CustomerID'][:20]

In [162]:
lookalike_results = {}

for customer_id in target_customers:
    customer_similarities = similarity_df[customer_id].sort_values(ascending=False)

    top_3_similar = customer_similarities.iloc[1:4].items()
    lookalike_results[customer_id] = [(other, score) for other, score in top_3_similar]

print(lookalike_results)

{'C0001': [('C0152', 0.9992287315057217), ('C0174', 0.9980419605334541), ('C0011', 0.9963622147816388)], 'C0002': [('C0134', 0.9947571543341317), ('C0007', 0.9929795154496197), ('C0159', 0.992738479637962)], 'C0003': [('C0031', 0.998471029972362), ('C0076', 0.9943179587408447), ('C0006', 0.9942735472304255)], 'C0004': [('C0108', 0.9973837694730558), ('C0099', 0.995915053857329), ('C0155', 0.9957870429039106)], 'C0005': [('C0159', 0.9999857771410672), ('C0007', 0.9989790256219125), ('C0002', 0.9924337889269538)], 'C0006': [('C0003', 0.9942735472304255), ('C0031', 0.9868622040025743), ('C0076', 0.9856195776904833)], 'C0007': [('C0159', 0.9991853163516102), ('C0005', 0.9989790256219125), ('C0002', 0.9929795154496197)], 'C0008': [('C0098', 0.9840332739839319), ('C0047', 0.9823728951162646), ('C0024', 0.9821540565897151)], 'C0009': [('C0197', 0.9922974204398337), ('C0061', 0.9901093096154499), ('C0132', 0.9843470505468875)], 'C0010': [('C0199', 0.9998202843853576), ('C0197', 0.9930828804691

In [164]:
lookalike_df = pd.DataFrame({'CustomerID': lookalike_results.keys(), 'Lookalikes': lookalike_results.values()})

In [166]:
lookalike_df.head()

,CustomerID,Lookalikes
0,C0001,"[(C0152, 0.9992287315057217), (C0174, 0.998041..."
1,C0002,"[(C0134, 0.9947571543341317), (C0007, 0.992979..."
2,C0003,"[(C0031, 0.998471029972362), (C0076, 0.9943179..."
3,C0004,"[(C0108, 0.9973837694730558), (C0099, 0.995915..."
4,C0005,"[(C0159, 0.9999857771410672), (C0007, 0.998979..."


In [168]:
lookalike_df['Lookalikes'][2] # checking, to see the output

[('C0031', 0.998471029972362),
 ('C0076', 0.9943179587408447),
 ('C0006', 0.9942735472304255)]

In [170]:
lookalike_df.to_csv('Lookalike.csv', index=False)
from IPython.display import FileLink
FileLink('Lookalike.csv')

/Users/adityagarg/Zeotap/Lookalike.csv